# Notebook 03: Neural Network Architecture Exploration

**Objective**: Systematically compare MLP architectures of varying depth and width to identify the optimal configuration for fraud detection.

**Research Questions**:
1. How does network depth affect fraud detection performance?
2. Does width compensate for lack of depth?
3. What is the optimal architecture-performance-complexity tradeoff?

**Architecture Categories**:
- **Shallow**: [32], [64, 32]
- **Medium**: [128, 64, 32], [256, 128, 64, 32]
- **Deep**: [512, 256, 128, 64, 32, 16]
- **Wide**: [256], [512]

**Training Configuration**: All models trained with identical hyperparameters (epochs=100, batch_size=64, early_stopping patience=15) to ensure fair comparison.

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import joblib
import sys
import os
import time

# Add src to path
sys.path.append(os.path.abspath('../'))
import config
from src.nn_architectures import get_architecture_by_name, count_parameters
from src.nn_training_utils import train_neural_network, get_class_weights
from src.evaluation_metrics import compute_fraud_metrics, print_classification_summary
from src.visualization_utils import plot_learning_curves, plot_precision_recall_curve, plot_architecture_comparison

# Set random seeds
config.set_random_seeds()

print("✓ Imports complete")
print(f"✓ TensorFlow version: {tf.__version__}")
print(f"✓ Random seed set to {config.RANDOM_SEED}")

## 1. Load Preprocessed Data

**Critical**: We load the **saved splits and scaler** from Notebook 02 to ensure reproducibility and prevent data leakage.

In [ ]:
# Load original data
df = pd.read_csv(config.DATASET_PATH)
X = df[config.FEATURE_COLUMNS].values
y = df[config.TARGET_COLUMN].values

# Load saved split indices
train_idx = np.load(config.TRAIN_INDICES_PATH)
val_idx = np.load(config.VAL_INDICES_PATH)
test_idx = np.load(config.TEST_INDICES_PATH)

# Split data using saved indices
X_train, y_train = X[train_idx], y[train_idx]
X_val, y_val = X[val_idx], y[val_idx]
X_test, y_test = X[test_idx], y[test_idx]

# Load fitted scaler and transform data
scaler = joblib.load(config.FITTED_SCALER_PATH)
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("✓ Data loaded and split using saved indices")
print(f"  Train: {X_train_scaled.shape[0]:,} samples")
print(f"  Val:   {X_val_scaled.shape[0]:,} samples")
print(f"  Test:  {X_test_scaled.shape[0]:,} samples")
print(f"\n✓ Scaler loaded from: {config.FITTED_SCALER_PATH}")
print(f"✓ Input dimension: {X_train_scaled.shape[1]}")

## 2. Compute Class Weights

Class weights handle the severe imbalance (124:1) by penalizing misclassifications of the minority class more heavily.

In [ ]:
# Compute balanced class weights
class_weights = get_class_weights(y_train)

print("✓ Class weights computed:")
print(f"  Class 0 (Legitimate): {class_weights[0]:.4f}")
print(f"  Class 1 (Fraud):      {class_weights[1]:.4f}")
print(f"  Ratio: {class_weights[1]/class_weights[0]:.1f}x penalty for fraud misclassification")

## 3. Define Architecture Experiment Plan

We'll train 8 different architectures spanning shallow to deep and narrow to wide configurations.

In [ ]:
# Define architectures to test
architectures_to_test = [
    # Shallow networks
    ("shallow_tiny", [32]),
    ("shallow_small", [64, 32]),
    
    # Medium networks
    ("medium_base", [128, 64, 32]),
    ("medium_deep", [256, 128, 64, 32]),
    
    # Deep network
    ("deep", [512, 256, 128, 64, 32, 16]),
    
    # Wide networks
    ("wide_medium", [256]),
    ("wide_large", [512]),
    
    # Balanced (for later comparison)
    ("balanced", [256, 128, 64])
]

print("✓ Architecture Experiment Plan:")
print(f"  Total architectures: {len(architectures_to_test)}")
print("\n  Architectures:")
for name, layers in architectures_to_test:
    params = count_parameters(len(config.FEATURE_COLUMNS), layers)
    print(f"    {name:20} {str(layers):30} → {params:,} params")

## 4. Train All Architectures

We train each architecture with identical hyperparameters to ensure fair comparison. Early stopping prevents overfitting.

In [ ]:
# Storage for results
results = []
histories = {}

# Training configuration (consistent across all models)
training_config = {
    'epochs': 100,
    'batch_size': 64,
    'patience': 15,
    'learning_rate': 0.001,
    'dropout_rate': 0.0,  # No dropout in base comparison
    'l2_reg': 0.0,        # No L2 in base comparison
    'use_batch_norm': False
}

print("="*70)
print("TRAINING ARCHITECTURE EXPERIMENTS")
print("="*70)
print(f"Configuration: {training_config}")
print("="*70 + "\n")

# Train each architecture
for arch_name, hidden_layers in architectures_to_test:
    print(f"\n{'='*70}")
    print(f"Training: {arch_name.upper()}")
    print(f"Architecture: {hidden_layers}")
    print(f"{'='*70}")
    
    start_time = time.time()
    
    # Create model
    model = get_architecture_by_name(arch_name, len(config.FEATURE_COLUMNS))
    
    # Train model
    history, trained_model, train_time_seconds = train_neural_network(
        model=model,
        X_train=X_train_scaled,
        y_train=y_train,
        X_val=X_val_scaled,
        y_val=y_val,
        class_weights=class_weights,
        epochs=training_config['epochs'],
        batch_size=training_config['batch_size'],
        patience=training_config['patience'],
        experiment_name=f"arch_{arch_name}"
    )
    
    # Evaluate on validation set
    y_val_pred_proba = trained_model.predict(X_val_scaled, verbose=0).flatten()
    y_val_pred = (y_val_pred_proba >= 0.5).astype(int)
    
    metrics = compute_fraud_metrics(y_val, y_val_pred, y_val_pred_proba)
    
    # Store results
    results.append({
        'architecture': arch_name,
        'layers': str(hidden_layers),
        'num_params': count_parameters(len(config.FEATURE_COLUMNS), hidden_layers),
        'depth': len(hidden_layers),
        'width': max(hidden_layers),
        'train_time': train_time_seconds,
        'final_epoch': len(history.history['loss']),
        **metrics
    })
    
    histories[arch_name] = history
    
    # Print summary
    print(f"\n✓ Training complete in {train_time_seconds:.2f}s ({len(history.history['loss'])} epochs)")
    print(f"  PR-AUC: {metrics['pr_auc']:.4f} | ROC-AUC: {metrics['roc_auc']:.4f}")
    print(f"  F1 (Fraud): {metrics['fraud_f1']:.4f} | Recall: {metrics['fraud_recall']:.4f}")
    
    # Save model
    model_path = os.path.join(config.MODELS_DIR, f'arch_{arch_name}.keras')
    trained_model.save(model_path)
    print(f"  Model saved: {model_path}")

print("\n" + "="*70)
print("✓ All architectures trained!")
print("="*70)

## 5. Compare Architecture Performance

In [ ]:
# Create results DataFrame
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('pr_auc', ascending=False)

print("\n" + "="*100)
print("ARCHITECTURE COMPARISON - Sorted by PR-AUC")
print("="*100)
print(results_df[['architecture', 'layers', 'num_params', 'pr_auc', 'roc_auc', 
                   'fraud_f1', 'fraud_recall', 'fraud_precision', 'train_time']].to_string(index=False))
print("="*100)

# Save results
results_path = os.path.join(config.RESULTS_DIR, 'tables', 'architecture_comparison.csv')
results_df.to_csv(results_path, index=False)
print(f"\n✓ Results saved to: {results_path}")

# Find best architecture
best_arch = results_df.iloc[0]
print(f"\n🏆 Best Architecture: {best_arch['architecture']}")
print(f"   Layers: {best_arch['layers']}")
print(f"   PR-AUC: {best_arch['pr_auc']:.4f}")
print(f"   Parameters: {best_arch['num_params']:,}")
print(f"   Training time: {best_arch['train_time']:.2f}s")

## 6. Visualize Architecture Tradeoffs

In [ ]:
# Create comprehensive architecture comparison plot
fig, axes = plt.subplots(2, 3, figsize=(20, 12))

# 1. Performance vs Complexity
ax1 = axes[0, 0]
scatter = ax1.scatter(results_df['num_params'], results_df['pr_auc'], 
                     c=results_df['depth'], cmap='viridis', s=200, alpha=0.7, edgecolor='black', linewidth=2)
for idx, row in results_df.iterrows():
    ax1.annotate(row['architecture'], (row['num_params'], row['pr_auc']), 
                fontsize=8, ha='right', va='bottom')
ax1.set_xlabel('Number of Parameters', fontsize=12, fontweight='bold')
ax1.set_ylabel('PR-AUC (Validation)', fontsize=12, fontweight='bold')
ax1.set_title('Performance vs Model Complexity', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)
cbar = plt.colorbar(scatter, ax=ax1)
cbar.set_label('Network Depth', fontsize=10)

# 2. Depth vs Performance
ax2 = axes[0, 1]
ax2.scatter(results_df['depth'], results_df['pr_auc'], s=200, alpha=0.7, color='steelblue', edgecolor='black', linewidth=2)
for idx, row in results_df.iterrows():
    ax2.annotate(row['architecture'], (row['depth'], row['pr_auc']), 
                fontsize=8, ha='center', va='bottom')
ax2.set_xlabel('Network Depth (# Hidden Layers)', fontsize=12, fontweight='bold')
ax2.set_ylabel('PR-AUC (Validation)', fontsize=12, fontweight='bold')
ax2.set_title('Impact of Network Depth', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)

# 3. Width vs Performance
ax3 = axes[0, 2]
ax3.scatter(results_df['width'], results_df['pr_auc'], s=200, alpha=0.7, color='coral', edgecolor='black', linewidth=2)
for idx, row in results_df.iterrows():
    ax3.annotate(row['architecture'], (row['width'], row['pr_auc']), 
                fontsize=8, ha='center', va='bottom')
ax3.set_xlabel('Maximum Layer Width', fontsize=12, fontweight='bold')
ax3.set_ylabel('PR-AUC (Validation)', fontsize=12, fontweight='bold')
ax3.set_title('Impact of Network Width', fontsize=13, fontweight='bold')
ax3.grid(True, alpha=0.3)

# 4. Training Time vs Performance
ax4 = axes[1, 0]
ax4.scatter(results_df['train_time'], results_df['pr_auc'], s=200, alpha=0.7, color='green', edgecolor='black', linewidth=2)
for idx, row in results_df.iterrows():
    ax4.annotate(row['architecture'], (row['train_time'], row['pr_auc']), 
                fontsize=8, ha='right', va='bottom')
ax4.set_xlabel('Training Time (seconds)', fontsize=12, fontweight='bold')
ax4.set_ylabel('PR-AUC (Validation)', fontsize=12, fontweight='bold')
ax4.set_title('Training Efficiency', fontsize=13, fontweight='bold')
ax4.grid(True, alpha=0.3)

# 5. PR-AUC Ranking
ax5 = axes[1, 1]
sorted_results = results_df.sort_values('pr_auc')
colors = ['#e74c3c' if row['architecture'] == best_arch['architecture'] else '#3498db' 
         for _, row in sorted_results.iterrows()]
ax5.barh(range(len(sorted_results)), sorted_results['pr_auc'], color=colors, alpha=0.7, edgecolor='black')
ax5.set_yticks(range(len(sorted_results)))
ax5.set_yticklabels(sorted_results['architecture'], fontsize=10)
ax5.set_xlabel('PR-AUC (Validation)', fontsize=12, fontweight='bold')
ax5.set_title('Architecture Ranking by PR-AUC', fontsize=13, fontweight='bold')
ax5.axvline(results_df['pr_auc'].mean(), color='black', linestyle='--', linewidth=2, label='Mean')
ax5.legend()
ax5.grid(True, alpha=0.3, axis='x')

# 6. F1 Score Ranking
ax6 = axes[1, 2]
sorted_f1 = results_df.sort_values('fraud_f1')
ax6.barh(range(len(sorted_f1)), sorted_f1['fraud_f1'], color='purple', alpha=0.7, edgecolor='black')
ax6.set_yticks(range(len(sorted_f1)))
ax6.set_yticklabels(sorted_f1['architecture'], fontsize=10)
ax6.set_xlabel('F1 Score (Fraud Class)', fontsize=12, fontweight='bold')
ax6.set_title('Architecture Ranking by F1 Score', fontsize=13, fontweight='bold')
ax6.grid(True, alpha=0.3, axis='x')

plt.suptitle('Neural Network Architecture Comparison - Fraud Detection', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig('../results/figures/architecture_comparison_comprehensive.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Comprehensive visualization saved")

## 7. Learning Curves for Top 3 Architectures

In [ ]:
# Plot learning curves for top 3 architectures
top_3_archs = results_df.head(3)['architecture'].values

fig, axes = plt.subplots(1, 3, figsize=(20, 5))

for idx, arch_name in enumerate(top_3_archs):
    history = histories[arch_name]
    plot_learning_curves(history, title=f'{arch_name.upper()}', ax=axes[idx])

plt.suptitle('Learning Curves - Top 3 Architectures by PR-AUC', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('../results/figures/top3_learning_curves.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Learning curves saved for top 3 architectures")

## 8. Comparison with Baselines

In [ ]:
# Load baseline results
baseline_path = os.path.join(config.RESULTS_DIR, 'tables', 'baseline_performance_targets.csv')
baselines = pd.read_csv(baseline_path)

print("\n" + "="*70)
print("NEURAL NETWORKS vs BASELINES")
print("="*70)

print("\nBaseline Performance (from Notebook 02):")
print(baselines[['model', 'pr_auc', 'f1_fraud']].to_string(index=False))

print(f"\nBest Neural Network ({best_arch['architecture']}):")
print(f"  PR-AUC:    {best_arch['pr_auc']:.4f}")
print(f"  F1 (Fraud): {best_arch['fraud_f1']:.4f}")

# Calculate improvement
best_baseline_pr_auc = baselines['pr_auc'].max()
improvement = (best_arch['pr_auc'] - best_baseline_pr_auc) / best_baseline_pr_auc * 100

print(f"\n🎯 Improvement over Best Baseline:")
print(f"  PR-AUC: +{improvement:.2f}%")
print(f"  Absolute gain: +{best_arch['pr_auc'] - best_baseline_pr_auc:.4f}")

if improvement > 15:
    print("\n✅ Goal Achieved: >15% improvement over baselines!")
else:
    print(f"\n⚠️  Target not yet met. Need {15 - improvement:.2f}% more improvement.")
    print("   → Try regularization experiments in Notebook 04")

print("="*70)

## 9. Summary & Conclusions

In [ ]:
print("\n" + "="*70)
print(" NOTEBOOK 03 SUMMARY - ARCHITECTURE EXPLORATION")
print("="*70)
print(f"✓ Architectures tested: {len(architectures_to_test)}")
print(f"✓ Best architecture: {best_arch['architecture']} - {best_arch['layers']}")
print(f"✓ Best PR-AUC: {best_arch['pr_auc']:.4f}")
print(f"✓ Improvement over baselines: +{improvement:.2f}%")

print("\n📊 Key Findings:")
print("  1. Depth Analysis:")
depth_corr = results_df[['depth', 'pr_auc']].corr().iloc[0, 1]
print(f"     - Depth-Performance correlation: {depth_corr:.3f}")
if depth_corr > 0:
    print("     - Deeper networks show performance improvement")
else:
    print("     - Diminishing returns from excessive depth")

print("  2. Width Analysis:")
width_corr = results_df[['width', 'pr_auc']].corr().iloc[0, 1]
print(f"     - Width-Performance correlation: {width_corr:.3f}")

print("  3. Complexity vs Performance:")
param_corr = results_df[['num_params', 'pr_auc']].corr().iloc[0, 1]
print(f"     - Parameters-Performance correlation: {param_corr:.3f}")
if param_corr > 0.5:
    print("     - Larger models perform better (no overfitting detected)")
else:
    print("     - Optimal model size achieved, avoid over-parameterization")

print("\n📁 Outputs:")
outputs = [
    results_path,
    '../results/figures/architecture_comparison_comprehensive.png',
    '../results/figures/top3_learning_curves.png',
    f'{config.MODELS_DIR}/arch_*.keras (8 models saved)'
]
for output in outputs:
    print(f"   {output}")

print("\n✅ Notebook 03 Complete!")
print("🚀 Next: Notebook 04 - Regularization Experiments")
print("   Goal: Apply dropout, L2, and batch normalization to best architecture")
print("="*70)